In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch


df = pd.read_csv('data/BTC-USD.csv')

# Indicate whether buy or sell
df['Future Close'] = df['Close'].shift(-1)
df['Target'] = df.apply(lambda row: 1 if row['Future Close'] > row['Close'] else 0, axis=1)

# Drop the last row since it does not have a future close price
df.dropna(inplace=True)

# Normalize the features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Volume']])


# Create Sequences
def create_sequences(features, target, sequence_length=3):
    X, y = [], []
    for i in range(len(features) - sequence_length):
        X.append(features[i:i + sequence_length])
        y.append(target.iloc[i + sequence_length])
    return np.array(X), np.array(y)



sequence_length = 5
X, y = create_sequences(features_scaled, df['Target'], sequence_length)


X = np.array(X)
y = np.array(y)

# Split the data into training, validation, and test sets
X_train, X_help, y_train, y_help = train_test_split(X, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_help, y_help, test_size=0.5, random_state=42)

# Convert back to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)

batch_size = 128
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
validation_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)



In [50]:

import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix

class GRU_NET(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU_NET, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)  
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input,hidden):
        output, hidden = self.gru(input, hidden)
        output = self.fc(output[:, -1, :])  # Use the output from the last time step
        output = self.sigmoid(output)
        return output,hidden

    def init_hidden(self, batch_size, device):
        return torch.zeros(1, batch_size, self.hidden_size).to(device)    


def train(model, train_loader, num_epochs=50):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in tqdm(train_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Initialize hidden state
            hidden = model.init_hidden(X_batch.size(0), device)

            # Forward pass
            outputs, hidden = model(X_batch,hidden)
            loss = criterion(outputs, y_batch.unsqueeze(1).float())  # Ensure y_batch is float for BCELoss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
        evaluate(model, validation_loader)
        
        


def predict(model, data, device):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        predictions = []
        hidden = model.init_hidden(1, device)  # Initialize hidden state
        for sample in data:
            sample = sample.unsqueeze(0)  # Add batch dimension
            output, hidden = model(sample, hidden)
            predictions.append(output.item())
    return predictions


def evaluate(model, val_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    with torch.no_grad():
        model.eval()
        correct = 0
        total = 0
        tp = 0
        tn = 0
        fp = 0
        fn = 0

        for input, labels in val_loader:
            input, labels = input.to(device), labels.to(device)
            hidden = model.init_hidden(input.size(0), device)  # Initialize hidden state
            outputs, hidden = model(input, hidden)
            predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
            tp += ((predicted == 1) & (labels.unsqueeze(1) == 1)).sum().item()
            tn += ((predicted == 0) & (labels.unsqueeze(1) == 0)).sum().item()
            fp += ((predicted == 1) & (labels.unsqueeze(1) == 0)).sum().item()
            fn += ((predicted == 0) & (labels.unsqueeze(1) == 1)).sum().item()

        print(f'Accuracy: {correct / total:.4f}')
        print(f'True Positive: {tp}')
        print(f'True Negative: {tn}')
        print(f'False Positive: {fp}')
        print(f'False Negative: {fn}')
            
        

In [51]:

model = GRU_NET(input_size=5, hidden_size=64, output_size=1)
train(model, train_loader)
predict(model, X_test, 'cuda:0')

print('-------------------------')
print('Evaluation on test set')
evaluate(model, test_loader)

100%|██████████| 13/13 [00:00<00:00, 48.63it/s]


Epoch [1/50], Loss: 0.6945
Accuracy: 0.5446
True Positive: 299
True Negative: 0
False Positive: 250
False Negative: 0


100%|██████████| 13/13 [00:00<00:00, 122.28it/s]


Epoch [2/50], Loss: 0.6903
Accuracy: 0.5446
True Positive: 299
True Negative: 0
False Positive: 250
False Negative: 0


100%|██████████| 13/13 [00:00<00:00, 152.43it/s]


...

Epoch [48/50], Loss: 0.6891
Accuracy: 0.5464
True Positive: 277
True Negative: 23
False Positive: 227
False Negative: 22


100%|██████████| 13/13 [00:00<00:00, 165.00it/s]


Epoch [49/50], Loss: 0.6892
Accuracy: 0.5428
True Positive: 282
True Negative: 16
False Positive: 234
False Negative: 17


100%|██████████| 13/13 [00:00<00:00, 89.89it/s]


Epoch [50/50], Loss: 0.6892
Accuracy: 0.5464
True Positive: 286
True Negative: 14
False Positive: 236
False Negative: 13
-------------------------
Evaluation on test set
Accuracy: 0.5420
True Positive: 277
True Negative: 20
False Positive: 232
False Negative: 19
